In [ ]:
# import libraries
import pandas as pd
import numpy as np
import datetime as dt

from sklearn.impute import SimpleImputer

from paths import RAW_DIR

In [ ]:
# functions

def add_col(data, to_add, to_remove):
    
    data[to_add] = np.where(data[to_remove].isna(), 0, 1)
    
    data = data.drop([to_remove], axis=1)
    return data

def replace_nan_1cond(data, feat1, val1, col_to_replace, val_to_replace_with):
    data.loc[(data[feat1] == val1), 
             col_to_replace] = val_to_replace_with
    return data

def replace_nan_2cond(data, feat1, feat2, val1, val2, col_to_replace, val_to_replace_with):
    data.loc[(data[feat1] == val1) & (data[feat2] == val2), 
             col_to_replace] = val_to_replace_with
    return data

### Import the data

In [ ]:
df1 = pd.read_excel(RAW_DIR / "breast_cancer_data.xlsx")
df2 = pd.read_excel(RAW_DIR / "breast_cancer_data_2.xlsx")

# concatenate the two datasets
data = pd.concat([df1, df2]).set_index('ehr')
data.head(20)

### Creating new columns

In [ ]:
data['age']= np.where(data['death_date'].isna(),
                      (dt.datetime.today()-pd.to_datetime(data['birth_date'])).astype('timedelta64[Y]'),
                      (pd.to_datetime(data['death_date'])-pd.to_datetime(data['birth_date'])).astype('timedelta64[Y]'))

In [ ]:
data['years_from_diagnosis']=(dt.datetime.today()\
            -pd.to_datetime(data['diagnosis_date'])).astype('timedelta64[Y]')

In [ ]:
data = add_col(data, 'dead', 'death_date')
data = add_col(data, 'recurrence', 'recurrence_year')
data = add_col(data, 'menopause', 'menopause_age')

### Drop columns

In [ ]:
data.isnull().sum()

In [ ]:
# drop birth_date, diagnosis_date, Unnamed: 0 (not useful) and side & caesarean (more than 200 null)
data = data.drop(['Unnamed: 0',
                  'birth_date',
                  'diagnosis_date',
                  'caesarean',
                  'side'], axis=1)
data.head(20)

In [ ]:
data.info()

### Data Visualization

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(14, 4))

sns.countplot(ax=axes[0], data=data, x="grade", hue="neoadjuvant", palette="RdPu")
axes[0].set(xlabel=None)
axes[0].set_title('Grade')

sns.countplot(ax=axes[1], data=data, x="hist_type", hue="neoadjuvant", palette="RdPu")
axes[1].set(xlabel=None)
axes[1].set_title('Histological Type')

sns.countplot(ax=axes[2], data=data, x="recurrence", hue="neoadjuvant", palette="RdPu")
axes[2].set(xlabel=None)
axes[2].set_title('Recurrence')

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(14, 4))

sns.violinplot(ax=axes[0], data=data, x="recurrence", y="ki67", hue="neoadjuvant", split=True, palette="Paired")
axes[0].set_title('Distribution of ki67')

sns.violinplot(ax=axes[1], data=data, x="recurrence", y="age", hue="neoadjuvant", split=True, palette="Paired")
axes[1].set_title('Distribution of Age')

sns.violinplot(ax=axes[2], data=data, x="recurrence", y="years_from_diagnosis", hue="neoadjuvant", split=True, palette="Paired")
axes[2].set_title('Distribution of years_from_diagnosis')

### Map values and replace with most frequent

In [ ]:
# checking the values of all the features using value counts
categoricals = ['neoadjuvant', 'hist_type']

for feature in categoricals:
    print(f"Values Counts for [{feature}]")
    print(data[feature].value_counts())
    print("\n")

In [ ]:
# 1 null value in neoadjuvant: probably is going to be solved when merging with the other dataset
data['neoadjuvant'] = data['neoadjuvant'].replace({'no': 0, 'yes': 1})
data['neoadjuvant'].value_counts()

In [ ]:
ordinal_col = ['hist_type']
data_dum = pd.get_dummies(data, columns = ordinal_col)

In [ ]:
# replace with most frequent value
for column in [
    #'neoadjuvant',   ### only 1 nan, maybe we find the true value with the merging
    'grade', 'invasive', 'er_positive', 'pr_positive', 'her2_positive', 'ki67', 'menarche_age']:
    data_dum[column].fillna(data_dum[column].mode()[0], inplace=True)

In [ ]:
data_dum.info()

In [ ]:
data_dum.groupby(['birth', 'pregnancy', 'abort'], dropna=False).size()

In [ ]:
# birth=-1 doesn't make sense --> replace it with 0

data_dum['birth'] = data_dum['birth'].replace({-1: 0})
data_dum.groupby(['birth', 'pregnancy', 'abort'], dropna=False).size()

In [ ]:
# replace abort Nan when birth = 0
data_dum = replace_nan_2cond(data_dum, 'birth', 'pregnancy', 0, 0, 'abort', 0)
data_dum = replace_nan_2cond(data_dum, 'birth', 'pregnancy', 0, 2, 'abort', 0)

# 98 cases with birth=0 and pregnancy,abort=NaN --> should we replace everything with 0?
#data_dum = replace_nan_1cond(data_dum, 'birth', 0, 'pregnancy', 0)
#data_dum = replace_nan_2cond(data_dum, 'birth', 'pregnancy', 0, 0, 'abort', 0)

# replace abort NaN when birth=1
data_dum = replace_nan_2cond(data_dum, 'birth', 'pregnancy', 1, 1, 'abort', 0)

# replace abort NaN when birth=2
data_dum = replace_nan_2cond(data_dum, 'birth', 'pregnancy', 2, 2, 'abort', 0)

# replace abort NaN when birth=3
data_dum = replace_nan_2cond(data_dum, 'birth', 'pregnancy', 3, 3, 'abort', 0)

# 1 case with birth=3, but pregnancy and abort NaN --> should we replace with pregnancy=3 and abort=0?
#data_dum = replace_nan_1cond(data_dum, 'birth', 3, 'pregnancy', 3)
#data_dum = replace_nan_2cond(data_dum, 'birth', 'pregnancy', 3, 3, 'abort', 0)

In [ ]:
data_dum.groupby(['birth', 'pregnancy', 'abort'], dropna=False).size()

In [ ]:
data_dum.info()